In [14]:
import requests
import pandas as pd
import json
import numpy as np
import requests
import html5lib
from bs4 import BeautifulSoup
import re
import time

In [15]:
URL="https://www.dlsu.edu.ph"

In [16]:
page = requests.get(URL)

# Extract all sub-urls from dlsu.edu.ph

In [ ]:


soup = BeautifulSoup(page.content, 'lxml')
data = []
links = []

url_limit = 50

start = time.time()

def remove_duplicates(l): # remove duplicates and unURL string
    for item in l:
        match = re.search("(?P<url>https?://[^\s]+)", item)
        if match is not None:
            links.append((match.group("url")))


for link in soup.find_all('a', href=True):
    data.append(str(link.get('href')))
    
flag = True
remove_duplicates(data)

while flag:
    try:
        for link in links:
            for j in soup.find_all('a', href=True):
                temp = []
                source_code = requests.get(link)
                soup = BeautifulSoup(source_code.content, 'lxml')
                temp.append(str(j.get('href')))
                remove_duplicates(temp)

                if len(links) > url_limit: # set limitation to number of URLs
                    break
            if len(links) > url_limit:
                break
        if len(links) > url_limit:
            break
    except Exception as e:
        print(e)
        if len(links) > url_limit:
            break
            
for url in links:
    print(url)
    
print ("Elapsed Time: %s" % (time.time() - start))

# NEW GET ALL INFORMATION

In [17]:
def deCFEmail(fp):
    try:
        r = int(fp[:2],16)
        email = ''.join([chr(int(fp[i:i+2], 16) ^ r) for i in range(2, len(fp), 2)])
        return email
    except (ValueError):
        pass

In [18]:
a = 'https://www.dlsu.edu.ph/colleges/ccs/office-of-the-dean/'
testpage = requests.get(a)
soup2 = BeautifulSoup(testpage.content, 'lxml')

table = soup2.find('div',{'class':'wpb_wrapper'})
table

<div class="wpb_wrapper">
<div class="wpb_text_column wpb_content_element">
<div class="wpb_wrapper">
<table cellpadding="0" cellspacing="0" summary="This table is a chart of the Office of the CCS Dean" width="90%">
<tbody>
<tr>
<td><strong>Dean</strong></td>
<td><a href="/cdn-cgi/l/email-protection#3240535453575e1c5153504057565d72565e41471c5756471c425a">Dr. Rafael A. Cabredo</a></td>
</tr>
<tr>
<td><b>Associate Dean</b></td>
<td><a href="/cdn-cgi/l/email-protection#6b08030a1902090e1f034508030e050c2b0f07181e450e0f1e451b03">Dr. Charibeth K. Cheng</a></td>
</tr>
<tr class="selected">
<td><strong>Assistant Dean, External Affairs and Lasallian Mission</strong></td>
<td><a href="/cdn-cgi/l/email-protection#15667d7c6779706c3b767d6055717966603b7071603b657d">Ms. Shirley B. Chu</a></td>
</tr>
<tr class="selected">
<td><strong>Assistant Dean, Research and Graduate Studies</strong></td>
<td><a href="/cdn-cgi/l/email-protection#46252e342f35322f282368222f2728236834272b293506222a35336823223368362e">

In [44]:
position = []
names = []
emails = []
department = []
urlsList = []

temp = ""
temp_dept = ""

dept = "Department"
titles = ["Mr","Dr","Mrs","Ms"]

table_length = len(table.find_all('td'))

for x in range(0,table_length):
    temp = table.find_all('td')[x]      
    
    if dept in temp.text:
        temp_dept = temp.text
    elif any(word in temp.text for word in titles):
        names.append(temp.text)
        department.append(temp_dept)
    elif temp.text == None:
        pass
    else:
        position.append(temp.text)
       
        
    try:
         # email
        temp2 = table.find_all('td')[x].a['href']
        if "email-protect" in temp2:
            emails.append(deCFEmail(temp2.split("#")[1]))
    except:
        pass
    
    
    

In [45]:
position = list(filter(None, position))

print(len(names))
print(len(urlsList))
print(len(department))
print(len(position))
print(len(emails))

department

23
0
23
23
23


['',
 '',
 '',
 '',
 '',
 '',
 'Department of Computer Technology\xa0',
 'Department of Computer Technology\xa0',
 'Department of Computer Technology\xa0',
 'Department of Computer Technology\xa0',
 'Department of Computer Technology\xa0',
 'Department of Information Technology\xa0',
 'Department of Information Technology\xa0',
 'Department of Information Technology\xa0',
 'Department of Information Technology\xa0',
 'Department of Information Technology\xa0',
 'Department of Software Technology\xa0',
 'Department of Software Technology\xa0',
 'Department of Software Technology\xa0',
 'Department of Software Technology\xa0',
 'Department of Software Technology\xa0',
 'Department of Software Technology\xa0',
 'Department of Software Technology\xa0']

In [46]:
position = list(filter(None, position))

testList = list(zip(names,position,emails,department))
testList

[('Dr. Rafael A. Cabredo', 'Dean', 'rafael.cabredo@dlsu.edu.ph', ''),
 ('Dr. Charibeth K. Cheng',
  'Associate Dean',
  'charibeth.cheng@dlsu.edu.ph',
  ''),
 ('Ms. Shirley B. Chu',
  'Assistant Dean, External Affairs and Lasallian Mission',
  'shirley.chu@dlsu.edu.ph',
  ''),
 ('Dr. Christine Diane L. Ramos',
  'Assistant Dean, Research and Graduate Studies',
  'christine.diane.ramos@dlsu.edu.ph',
  ''),
 ('Dr. Ma. Rowena R. Caguiat',
  'Director, Consulting and Education Center',
  'ma.rowena.caguiat@dlsu.edu.ph',
  ''),
 ('Mr. Gregory G. Cu',
  'Head, Technical Support Group',
  'gregory.cu@dlsu.edu.ph',
  ''),
 ('Dr. Marnel S. Peradilla',
  'Chair',
  'marnel.peradilla@dlsu.edu.ph',
  'Department of Computer Technology\xa0'),
 ('Ms. Arlyn Verina L. Ong-Tiu',
  'Vice Chair',
  'arlyn.ong@dlsu.edu.ph',
  'Department of Computer Technology\xa0'),
 ('Dr. Ronald M. Pascual',
  'Coordinator, CT Thesis',
  'ronald.pascual@dlsu.edu.ph',
  'Department of Computer Technology\xa0'),
 ('Ms. Ar

# Convert to txt file

In [48]:
df = pd.DataFrame (testList, columns = ['Name','Position', 'Email', 'Department'])
np.savetxt(r'np.txt', df.values, fmt='%s %s %s %s')

# Get Name & Email OLD

In [6]:
emailList = []
nameList = []
urlList = []

def emailNameExtractor(urlString):
    getH=requests.get(urlString)
    h=getH.content
    soup=BeautifulSoup(h,'html.parser')
    mailtos = soup.select('a[href]')
    for i in mailtos:
        href=i['href']
        if "email-protect" in href:
            emailList.append(deCFEmail(href.split("#")[1]))
            nameList.append(i.text)
            urlList.append(urlString)

In [7]:
a = 'https://www.dlsu.edu.ph/colleges/ccs/office-of-the-dean/'
links.append(a)

In [8]:
emailList = []
nameList = []
urlFrom = []

length = 0
prev_length =  0



for x in range(len(links)):
    prev_length = length
    length = len(emailList)
    
    if prev_length != length: 
        print("Email Length: ",len(emailList), "Name Length: ", len(nameList), "index: ", x)
        
    
    print(links[x])
    
    if links[x].lower().endswith(".pdf"):
        pass
    else:
        try:
            emailNameExtractor(links[x])
        except:
            pass
   
            
emailList
nameList

https://www.dlsu.edu.ph/about-dlsu/
https://www.dlsu.edu.ph/about-dlsu/ourcorevalues/
https://www.dlsu.edu.ph/offices/qao/accreditation/
Email Length:  1 Name Length:  1 index:  3
https://www.dlsu.edu.ph/offices/office-of-the-chancellor/
Email Length:  2 Name Length:  2 index:  4
https://www.dlsu.edu.ph/university-fellows/
https://www.dlsu.edu.ph/emeriti/
https://www.dlsu.edu.ph/brothers/
https://www.dlsu.edu.ph/lasallianmission/
https://www.dlsu.edu.ph/offices/
https://www.dlsu.edu.ph/inside/board-advisors/
https://www.dlsu.edu.ph/admissions/scholarships/
Email Length:  3 Name Length:  3 index:  11
https://www.dlsu.edu.ph/admissions/graduate/
Email Length:  5 Name Length:  5 index:  12
https://www.dlsu.edu.ph/integrated-school/
https://www.dlsu.edu.ph/integrated-school/
https://www.dlsu.edu.ph/integrated-school/
https://www.dlsu.edu.ph/integratedschool-test/senior-high-school/
Email Length:  11 Name Length:  11 index:  16
https://www.dlsu.edu.ph/colleges/
https://www.dlsu.edu.ph/colle

https://www.dlsu.edu.ph/contact-us-new-normal/
Email Length:  54 Name Length:  54 index:  121
https://www.dlsu.edu.ph/contact-us-new-normal/
Email Length:  69 Name Length:  69 index:  122
https://www.dlsu.edu.ph/dlsu-care-desk/
https://www.dlsu.edu.ph/dlsu-care-desk/
https://www.dlsu.edu.ph/covid-19-resource/
https://www.dlsu.edu.ph/covid-19-resource/
https://www.dlsu.edu.ph/connectivity-support/
https://www.dlsu.edu.ph/connectivity-support/
https://www.dlsu.edu.ph/dlsu-return-to-campus-guidelines/
https://www.dlsu.edu.ph/dlsu-return-to-campus-guidelines/
https://www.dlsu.edu.ph/lasallians-r-e-a-c-h/
https://www.dlsu.edu.ph/lasallians-r-e-a-c-h/
https://www.dlsu.edu.ph/list-of-important-help-desk-announcements/
https://www.dlsu.edu.ph/list-of-important-help-desk-announcements/
https://www.dlsu.edu.ph/give/
https://www.dlsu.edu.ph/animo-christmas-2022/
https://www.dlsu.edu.ph/animo-christmas-2022/
https://www.dlsu.edu.ph/online-request-for-documents/
https://www.dlsu.edu.ph/online-reque

['[email\xa0protected]',
 '[email\xa0protected]',
 '[email\xa0protected]',
 '[email\xa0protected]',
 '[email\xa0protected]',
 '[email\xa0protected]',
 '[email\xa0protected]',
 '[email\xa0protected]',
 '[email\xa0protected]',
 '[email\xa0protected]',
 '[email\xa0protected]',
 '[email\xa0protected]',
 '[email\xa0protected]',
 '[email\xa0protected]',
 '[email\xa0protected]',
 '[email\xa0protected]',
 '[email\xa0protected]',
 '[email\xa0protected]',
 '[email\xa0protected]',
 '[email\xa0protected]',
 '[email\xa0protected]',
 '[email\xa0protected]',
 '[email\xa0protected]',
 '[email\xa0protected]',
 '[email\xa0protected]',
 '[email\xa0protected]',
 '[email\xa0protected]',
 '[email\xa0protected]',
 '[email\xa0protected]',
 '[email\xa0protected]',
 '[email\xa0protected]',
 '[email\xa0protected]',
 '[email\xa0protected]',
 '[email\xa0protected]',
 '[email\xa0protected]',
 '[email\xa0protected]',
 '[email\xa0protected]',
 '[email\xa0protected]',
 '[email\xa0protected]',
 '[email\xa0protected]',


# Combined Email and Name List OLD

In [19]:
urlList

['https://www.dlsu.edu.ph/offices/qao/accreditation/',
 'https://www.dlsu.edu.ph/offices/office-of-the-chancellor/',
 'https://www.dlsu.edu.ph/admissions/graduate/',
 'https://www.dlsu.edu.ph/admissions/graduate/',
 'https://www.dlsu.edu.ph/integratedschool-test/senior-high-school/',
 'https://www.dlsu.edu.ph/integratedschool-test/senior-high-school/',
 'https://www.dlsu.edu.ph/integratedschool-test/senior-high-school/',
 'https://www.dlsu.edu.ph/integratedschool-test/senior-high-school/',
 'https://www.dlsu.edu.ph/integratedschool-test/senior-high-school/',
 'https://www.dlsu.edu.ph/integratedschool-test/senior-high-school/',
 'https://www.dlsu.edu.ph/offices/hub/cav-ched-requests/',
 'https://www.dlsu.edu.ph/research/research-centers/shore/',
 'https://www.dlsu.edu.ph/research/research-centers/shore/',
 'https://www.dlsu.edu.ph/research/research-centers/bnscwc/',
 'https://www.dlsu.edu.ph/research/research-centers/bnscwc/',
 'https://www.dlsu.edu.ph/research/research-centers/cesdr/',

In [ ]:
finalList = list(zip(nameList,emailList,urlList))
finalList = set(finalList)
finalList

# Threading 

## wala pang ginagawa to

In [12]:
import threading
import time

start = time.time()

def fetch_url(url):
    print ("'%s\' fetched in %ss" % (url, (time.time() - start)))
    
threads = [threading.Thread(target=fetch_url, args=(url,)) for url in links]

for thread in threads:
    thread.start()
    
for thread in threads:
    thread.join()

print ("Elapsed Time: %s" % (time.time() - start))

'https://www.dlsu.edu.ph/offices/' fetched in 0.0020012855529785156s
'https://www.dlsu.edu.ph/soll/' fetched in 0.004004478454589844s
'https://www.facebook.com/DLSU.GradStudies' fetched in 0.007005929946899414s'https://www.dlsu.edu.ph/give/' fetched in 0.008007287979125977s

'https://www.dlsu.edu.ph/research/research-centers/cesdr/' fetched in 0.01101827621459961s
'https://www.dlsu.edu.ph/research/research-centers/' fetched in 0.013028621673583984s'https://www.linkedin.com/school/de-la-salle-university/' fetched in 0.015024662017822266s

'https://www.dlsu.edu.ph/shs-research-congress/%20' fetched in 0.017004728317260742s
'https://www.dlsu.edu.ph/colleges/ccs/office-of-the-dean/' fetched in 0.02000141143798828s
'https://twitter.com/DLSUGradStudies' fetched in 0.022000789642333984s'https://www.dlsu.edu.ph/wp-content/uploads/2022/06/the-concierge.pdf' fetched in 0.023999929428100586s

'https://www.dlsu.edu.ph/research/research-centers/bnscwc/' fetched in 0.026002883911132812s
'https://www

# Get emails protected by cloudflare

# Pwede na ata di isama to kasi ayaw gumana

In [13]:
mydivs = soup.findAll("span",{"class": "__cf_email__"})

length = len(mydivs)

for x in range(length):
    a = cfDecodeEmail(mydivs[x]["data-cfemail"])
    print(a)


NameError: name 'cfDecodeEmail' is not defined

In [ ]:
print (mydivs[1]["data-cfemail"])
print(len(mydivs))